### import

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install numpy
!pip install gluonnlp==0.10.0
!pip install pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers
!pip install torch
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661750 sha256=702d1cea81d5314bb193122bd320677c6b94b475e697f368da8702630081e721
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 2.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metad

## 코드

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-eansfbcm/kobert-tokenizer_66e0946a68c849fbab9b7dfa041ece97
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-eansfbcm/kobert-tokenizer_66e0946a68c849fbab9b7dfa041ece97
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=1dabef5ca0320f5588747c2af77655097f80bcf7e58746f43db5168b258bb0d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-l0mn6ou_/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = '1'

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
np.bool = np.bool_
import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용시
device = torch.device("cuda:0")
# device = torch.device("cpu")

def get_kobert_model(model_path, vocab_file, ctx="cpu"):
  bertmodel = BertModel.from_pretrained(model_path)
  device = torch.device(ctx)
  bertmodel.to(device)
  bertmodel.eval()
  vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file, padding_token='[PAD]')
  return bertmodel, vocab_b_obj

#koBERT
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained("skt/kobert-base-v1")
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
tokenizer.encode("한국어 모델을 공유합니다.")

[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [ ]:
# get dataset
datasetURL = "https://raw.githubusercontent.com/IDIOcoder/Chat-bot/main/dataset/intent_dataset.csv"
data = pd.read_csv(datasetURL, encoding='utf-8')

data_list = []
for ques, label in zip(data['sentence'], data['intent']):
  data = []
  data.append(ques)
  data.append(str(label))

  data_list.append(data)

In [ ]:
#데이터가 잘 저장되었는지 확인
print(len(data_list))
print(data_list[0])

5450
['10분마다 연락을 해봤지. 근데 아직도 이녀석, 온다고만 하지 오질 않고 있어.', '0']


### 클래스 가중치 계산

In [ ]:
from collections import Counter

# 데이터셋의 라벨을 모두 모아 클래스 빈도 계산
labels = [int(label) for _, label in data_list]
class_counts = Counter(labels)
num_samples = len(labels)

# 클래스 가중치 계산 (클래스 빈도의 역수)
class_weights = {cls: num_samples / count for cls, count in class_counts.items()}
class_weights_tensor = torch.tensor([class_weights[i] for i in sorted(class_weights.keys())], dtype=torch.float).to(device)

### 입력 데이터셋 토큰화

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#Setting Parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

### KoBERT 모델 구현

In [ ]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size=768,
               num_classes=2,
               dr_rate=None,
               params=None):
    super(BERTClassifier, self).__init__()
    self.bert=bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v]=1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

#optimizer와 schedule설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params':[p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params':[p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor) #다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### 훈련

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))


<ipython-input-15-0cd8b201d8af>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/137 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7185406684875488 train acc 0.4375
epoch 1 train acc 0.8980383211678832


<ipython-input-15-0cd8b201d8af>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/35 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9991071428571429


  0%|          | 0/137 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.000136730435770005 train acc 1.0
epoch 5 train acc 1.0


  0%|          | 0/35 [00:00<?, ?it/s]

epoch 5 test acc 0.9991071428571429


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("대화의")
            elif np.argmax(logits) == 1:
                test_eval.append("레시피 요청의")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 의도가 느껴집니다.")

In [ ]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 스테이플러 만드는 법이 궁금하네요
>> 입력하신 내용에서 레시피 요청의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 자전거 만드는 법
>> 입력하신 내용에서 레시피 요청의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 서랍 만드는 법
>> 입력하신 내용에서 레시피 요청의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 안녕하세요
>> 입력하신 내용에서 레시피 요청의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 반가워요
>> 입력하신 내용에서 레시피 요청의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 이건 그렇게 하는 것이 아닙니다.
>> 입력하신 내용에서 대화의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 강지님은 진자 걸크러시
>> 입력하신 내용에서 레시피 요청의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘은 뭘 해볼까?
>> 입력하신 내용에서 대화의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 주변에서 이성만 보면 차갑다는 얘기를 듣습니다.
>> 입력하신 내용에서 대화의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 그럴 수 있지
>> 입력하신 내용에서 대화의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 걔네를 잠재적 여친으로 생각해서 그래
>> 입력하신 내용에서 대화의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 안녕하세요?
>> 입력하신 내용에서 대화의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 안녕하세요
>> 입력하신 내용에서 레시피 요청의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : exit
>> 입력하신 내용에서 레시피 요청의 의도가 느껴집니다.


하고싶은 말을 입력해주세요 : 0


In [ ]:
torch.save(model.state_dict(), 'intent_weights.pth')

In [ ]:
from google.colab import files

# 가중치 파일 다운로드
files.download('intent_weights.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>